In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
amzn_ratings = pd.read_csv('ratings_Beauty.csv')

In [4]:
amzn_ratings.head()

,UserId,ProductId,Rating,Timestamp
0,A39HTATAQ9V7YF,0205616461,5.0,1369699200
1,A3JM6GV9MNOF9X,0558925278,3.0,1355443200
2,A1Z513UWSAAO0F,0558925278,5.0,1404691200
3,A1WMRR494NWEWV,0733001998,4.0,1382572800
4,A3IAAVS479H7M7,0737104473,1.0,1274227200


In [5]:
amzn_ratings = amzn_ratings.sample(2000)    # get sample data randomly (specify sample size)

In [7]:
amzn_ratings.shape

(2000, 4)

In [8]:
unique_users = amzn_ratings['UserId'].unique()  # ensure every user ID is unique
print(len(unique_users))

1991


In [10]:
unique_products = amzn_ratings['ProductId'].unique()
print(len(unique_products))

1892


In [11]:
new_df = pd.DataFrame(unique_users)
new_df.columns = ['UserId']

In [12]:
for product in unique_products:
    new_df[product] = None  # make the whole product column in the dataframe as None

C:\Users\admin\AppData\Local\Temp\ipykernel_13532\2091099746.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df[product] = None  # make the whole product column in the dataframe as None
C:\Users\admin\AppData\Local\Temp\ipykernel_13532\2091099746.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df[product] = None  # make the whole product column in the dataframe as None
C:\Users\admin\AppData\Local\Temp\ipykernel_13532\2091099746.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result

In [13]:
# transform the dataframe to the form as shown by the output below
for j, user in enumerate(new_df['UserId'].values):
    for i in range(amzn_ratings[amzn_ratings['UserId'] == user].shape[0]):
        product_id = amzn_ratings[amzn_ratings['UserId'] == user]['ProductId'].iloc[i]
        rating = amzn_ratings[amzn_ratings['UserId'] == user]['Rating'].iloc[i]
        new_df[product_id][j] = rating

In [14]:
new_df.sample(5)

,UserId,B000TW2JQC,B00021DVCQ,B008E6K6MO,B0018C3V5A,B000I1TNB8,B00KWFDBKE,B001CD31XM,B004SU886Q,B001SERK9G,...,B004TMIBIS,B00HK0VA9I,B002KP1WIW,B00B594NEY,B001ET72C0,B003TNVSTW,B004PCJW7G,B00AAG9M2Q,B002AV0Z2A,B00I4ZTFCM
1610,AGS6KYWZM3ACK,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
25,A1WN5KH0202QH8,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
665,AKN8HDO4FMSPZ,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1646,A1J0GKZ30VYPR9,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1319,ALO8QYO3R0GBS,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [15]:
user = pd.DataFrame(new_df.iloc[1789]) # randomly select user
user = user.drop(['UserId'])
user[user.notnull().values]

,1789
B008BDQKQQ,4.0


In [16]:
subset = new_df[new_df[user[user.notnull().values].index[0]].notnull()]
subset.head()

,UserId,B000TW2JQC,B00021DVCQ,B008E6K6MO,B0018C3V5A,B000I1TNB8,B00KWFDBKE,B001CD31XM,B004SU886Q,B001SERK9G,...,B004TMIBIS,B00HK0VA9I,B002KP1WIW,B00B594NEY,B001ET72C0,B003TNVSTW,B004PCJW7G,B00AAG9M2Q,B002AV0Z2A,B00I4ZTFCM
1789,A3GRTX7KQFA3PM,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [17]:
subset = subset.replace([None], 0)
subset = subset.set_index('UserId')

In [18]:
subset.head()   # use tail() for row data counting upwords from bottom

,B000TW2JQC,B00021DVCQ,B008E6K6MO,B0018C3V5A,B000I1TNB8,B00KWFDBKE,B001CD31XM,B004SU886Q,B001SERK9G,B004TTXMQ2,...,B004TMIBIS,B00HK0VA9I,B002KP1WIW,B00B594NEY,B001ET72C0,B003TNVSTW,B004PCJW7G,B00AAG9M2Q,B002AV0Z2A,B00I4ZTFCM
UserId,,,,,,,,,,,,,,,,,,,,,
A3GRTX7KQFA3PM,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
subset['cos_similarity'] = 0.0
for user in subset.index:
    cos = cosine_similarity(subset.loc['A3GRTX7KQFA3PM'].values.reshape(1, -1), subset.loc[user].values.reshape(1, -1)) # use reshape() to reshape the data frame (reshape(1, -1) => upside down)

In [21]:
subset['cos_similarity']

UserId
A3GRTX7KQFA3PM    0.0
Name: cos_similarity, dtype: float64